In [2]:
!pip install weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.2/378.2 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.8/223.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 

In [3]:
!pip install weaviate-client transformers sentence-transformers torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 13.4 MB/s eta 0:00:00


In [4]:
import weaviate

# Replace with your Weaviate URL and API Key
WEAVIATE_CLUSTER='https://8bzv2af3rlyyuwwuqlviww.c0.asia-southeast1.gcp.weaviate.cloud'
WEAVIATE_API_KEY='uSSvy4HZRjfimTGKTEnF6CF0coa4UTEnregi'

# Initialize Weaviate client
client = weaviate.Client(
    url=WEAVIATE_CLUSTER,
    additional_headers={"Authorization": "Bearer " + WEAVIATE_API_KEY}
)

<ipython-input-4-5a73aef995ed>:8: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(


In [6]:
# Define schema
schema = {
    "classes": [
        {
            "class": "JourneySegment",
            "description": "Details of each flight segment in the journey",
            "properties": [
                {"name": "flight_number", "dataType": ["text"]},
                {"name": "source", "dataType": ["text"]},
                {"name": "destination", "dataType": ["text"]},
                {"name": "departure_date", "dataType": ["date"]},
                {"name": "arrival_date", "dataType": ["date"]},
                {"name": "layover_duration", "dataType": ["text"]},
                {"name": "passengers", "dataType": ["text"]}
            ],
            "vectorizer": "none"  # Using custom embeddings
        }
    ]
}

# Create schema if it doesn't exist
if not client.schema.exists("JourneySegment"):
    client.schema.create(schema)

In [7]:
import json # Import the json module

with open('/content/Journey_Details.json', 'r') as file:
    data = json.load(file)

In [8]:
data

{'user': {'id': 227,
  'flights': [{'ticket_id': 3183,
    'pnr': 'HZAVJJ',
    'class': 'ECONOMY',
    'source': 'Cape Town International Airport (CPT)',
    'destination': 'Indira Gandhi International Airport (DEL)',
    'departure_date': '2024-07-11T14:35:00.000Z',
    'arrival_date': '2024-07-12T08:10:00.000Z',
    'layover_duration': '55m',
    'segments': [{'flight_number': 'ET846',
      'departure': {'airport': 'Cape Town International Airport',
       'iata': 'CPT',
       'date': '2024-07-11T14:35:00.000Z'},
      'arrival': {'airport': 'Addis Ababa Bole International Airport',
       'iata': 'ADD',
       'date': '2024-07-11T22:00:00.000Z'},
      'passengers': [{'first_name': 'surendra',
        'last_name': 'singh',
        'seat_number': '21a',
        'cabin_baggage': '7kg',
        'check_in_baggage': '23kg'},
       {'first_name': 'narinder',
        'last_name': 'kaur',
        'seat_number': '21b',
        'cabin_baggage': '7kg',
        'check_in_baggage': '23kg'},


In [9]:
text_data = json.dumps(data, indent=4)

In [10]:
!pip install langchain
from langchain.docstore.document import Document

In [11]:
document = Document(page_content=text_data)

In [12]:
document

Document(metadata={}, page_content='{\n    "user": {\n        "id": 227,\n        "flights": [\n            {\n                "ticket_id": 3183,\n                "pnr": "HZAVJJ",\n                "class": "ECONOMY",\n                "source": "Cape Town International Airport (CPT)",\n                "destination": "Indira Gandhi International Airport (DEL)",\n                "departure_date": "2024-07-11T14:35:00.000Z",\n                "arrival_date": "2024-07-12T08:10:00.000Z",\n                "layover_duration": "55m",\n                "segments": [\n                    {\n                        "flight_number": "ET846",\n                        "departure": {\n                            "airport": "Cape Town International Airport",\n                            "iata": "CPT",\n                            "date": "2024-07-11T14:35:00.000Z"\n                        },\n                        "arrival": {\n                            "airport": "Addis Ababa Bole International Airp

In [13]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

In [14]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents([document])

In [15]:
!pip install sentence-transformers

In [16]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00


In [17]:
#creating embedding
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  #model_kwargs=model_kwargs
)

<ipython-input-17-f890f276ac26>:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
from langchain.vectorstores import Weaviate

In [20]:
vector_db = Weaviate.from_documents(
    docs, embeddings, client=client, by_text=False
)

In [21]:
print(vector_db.similarity_search("what is the seat_number of samik", k=1)[0].page_content)

"seat_number": "21c",
                                "cabin_baggage": "7kg",
                                "check_in_baggage": "23kg"
                            }
                        ]
                    },
                    {
                        "flight_number": "ET686",
                        "departure": {
                            "airport": "Addis Ababa Bole International Airport",
                            "iata": "ADD",
                            "date": "2024-07-11T22:55:00.000Z"
                        },
                        "arrival": {
                            "airport": "Indira Gandhi International Airport",
                            "iata": "DEL",
                            "date": "2024-07-12T08:10:00.000Z"
                        },
                        "passengers": [
                            {
                                "first_name": "surendra",


In [22]:
print(vector_db.similarity_search("where is the flight going all details ?", k=1)[0].page_content)

{
    "user": {
        "id": 227,
        "flights": [
            {
                "ticket_id": 3183,
                "pnr": "HZAVJJ",
                "class": "ECONOMY",
                "source": "Cape Town International Airport (CPT)",
                "destination": "Indira Gandhi International Airport (DEL)",
                "departure_date": "2024-07-11T14:35:00.000Z",
                "arrival_date": "2024-07-12T08:10:00.000Z",
                "layover_duration": "55m",
                "segments": [
                    {
                        "flight_number": "ET846",
                        "departure": {
                            "airport": "Cape Town International Airport",
                            "iata": "CPT",
                            "date": "2024-07-11T14:35:00.000Z"
                        },
                        "arrival": {
                            "airport": "Addis Ababa Bole International Airport",
                            "iata": "ADD",


In [24]:
print(vector_db.similarity_search("tell me the flight details?", k=1)[0].page_content)

{
    "user": {
        "id": 227,
        "flights": [
            {
                "ticket_id": 3183,
                "pnr": "HZAVJJ",
                "class": "ECONOMY",
                "source": "Cape Town International Airport (CPT)",
                "destination": "Indira Gandhi International Airport (DEL)",
                "departure_date": "2024-07-11T14:35:00.000Z",
                "arrival_date": "2024-07-12T08:10:00.000Z",
                "layover_duration": "55m",
                "segments": [
                    {
                        "flight_number": "ET846",
                        "departure": {
                            "airport": "Cape Town International Airport",
                            "iata": "CPT",
                            "date": "2024-07-11T14:35:00.000Z"
                        },
                        "arrival": {
                            "airport": "Addis Ababa Bole International Airport",
                            "iata": "ADD",


In [28]:
print(vector_db.similarity_search("tell me the seat_number, baggage, destination?", k=1)[0].page_content)

"seat_number": "21c",
                                "cabin_baggage": "7kg",
                                "check_in_baggage": "23kg"
                            }
                        ]
                    },
                    {
                        "flight_number": "ET686",
                        "departure": {
                            "airport": "Addis Ababa Bole International Airport",
                            "iata": "ADD",
                            "date": "2024-07-11T22:55:00.000Z"
                        },
                        "arrival": {
                            "airport": "Indira Gandhi International Airport",
                            "iata": "DEL",
                            "date": "2024-07-12T08:10:00.000Z"
                        },
                        "passengers": [
                            {
                                "first_name": "surendra",


In [29]:
print(vector_db.similarity_search("tell me the date?", k=1)[0].page_content)

"date": "2024-07-11T22:00:00.000Z"
                        },
                        "passengers": [
                            {
                                "first_name": "surendra",
                                "last_name": "singh",
                                "seat_number": "21a",
                                "cabin_baggage": "7kg",
                                "check_in_baggage": "23kg"
                            },
                            {
                                "first_name": "narinder",
                                "last_name": "kaur",
                                "seat_number": "21b",
                                "cabin_baggage": "7kg",
                                "check_in_baggage": "23kg"
                            },
                            {
                                "first_name": "samik",
                                "last_name": "singh",


In [30]:
print(vector_db.similarity_search("tell me the destination?", k=1)[0].page_content)

{
    "user": {
        "id": 227,
        "flights": [
            {
                "ticket_id": 3183,
                "pnr": "HZAVJJ",
                "class": "ECONOMY",
                "source": "Cape Town International Airport (CPT)",
                "destination": "Indira Gandhi International Airport (DEL)",
                "departure_date": "2024-07-11T14:35:00.000Z",
                "arrival_date": "2024-07-12T08:10:00.000Z",
                "layover_duration": "55m",
                "segments": [
                    {
                        "flight_number": "ET846",
                        "departure": {
                            "airport": "Cape Town International Airport",
                            "iata": "CPT",
                            "date": "2024-07-11T14:35:00.000Z"
                        },
                        "arrival": {
                            "airport": "Addis Ababa Bole International Airport",
                            "iata": "ADD",


In [32]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [33]:
prompt=ChatPromptTemplate.from_template(template)

In [34]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.\nUse the following pieces of retrieved context to answer the question.\nIf you don't know the answer, just say that you don't know.\nUse ten sentences maximum and keep the answer concise.\nQuestion: {question}\nContext: {context}\nAnswer:\n"), additional_kwargs={})])

In [35]:
from langchain import HuggingFaceHub

In [37]:
from google.colab import userdata
huggingfacehub_api_token=userdata.get('HuGGINGFACE_TOKEN')

In [38]:
model = HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":1, "max_length":180}
)

<ipython-input-38-cc8e38a048ba>:1: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  model = HuggingFaceHub(


In [39]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [40]:
retriever=vector_db.as_retriever()

In [41]:
output_parser=StrOutputParser()

In [42]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [43]:
print(vector_db.similarity_search("tell me the flight details?", k=1)[0].page_content)

{
    "user": {
        "id": 227,
        "flights": [
            {
                "ticket_id": 3183,
                "pnr": "HZAVJJ",
                "class": "ECONOMY",
                "source": "Cape Town International Airport (CPT)",
                "destination": "Indira Gandhi International Airport (DEL)",
                "departure_date": "2024-07-11T14:35:00.000Z",
                "arrival_date": "2024-07-12T08:10:00.000Z",
                "layover_duration": "55m",
                "segments": [
                    {
                        "flight_number": "ET846",
                        "departure": {
                            "airport": "Cape Town International Airport",
                            "iata": "CPT",
                            "date": "2024-07-11T14:35:00.000Z"
                        },
                        "arrival": {
                            "airport": "Addis Ababa Bole International Airport",
                            "iata": "ADD",


In [44]:
print(vector_db.similarity_search("tell me the date?", k=1)[0].page_content)

"date": "2024-07-11T22:00:00.000Z"
                        },
                        "passengers": [
                            {
                                "first_name": "surendra",
                                "last_name": "singh",
                                "seat_number": "21a",
                                "cabin_baggage": "7kg",
                                "check_in_baggage": "23kg"
                            },
                            {
                                "first_name": "narinder",
                                "last_name": "kaur",
                                "seat_number": "21b",
                                "cabin_baggage": "7kg",
                                "check_in_baggage": "23kg"
                            },
                            {
                                "first_name": "samik",
                                "last_name": "singh",
